In [24]:
import polars as pl
import sys
sys.path.append("../../")
from VGG.dataset import VGGDataset
from VGG.model import vgg16
from VGG.utils import DataCsvParser
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from pathlib import Path
import os
from os.path import join
from torch.utils.data import DataLoader
from glob import glob

In [47]:
csv_path= "../../datasets/COVID/covid_data.csv"

In [49]:
df = pl.read_csv(csv_path)
df = df.with_columns(
    pl.col("path").map_elements(
        lambda x: x.replace("katsuji/project/", "katsujitakeda/project/study/"),
        return_dtype=str
    )
)
df.write_csv("../../datasets/COVID/covid_data_ncc.csv")

In [50]:
df["path"][0]

'/home/katsujitakeda/project/study/kmodels/datasets/COVID/images/2020.01.24.919183-p27-132.png'

In [51]:
df = df.with_columns(
    pl.Series(range(len(df))).alias("new_file_name").cast(pl.String),
    pl.col("file_name").map_elements(lambda x: "."+x.split(".")[-1], return_dtype=str).alias("suffix"),
)

In [52]:
plus = (df["path"].map_elements(lambda x: str(Path(x).parent), return_dtype=str) + "/" +df["new_file_name"] + df["suffix"]).alias("new_file_path")
plus = pl.DataFrame(plus)
plus[0].item()

'/home/katsujitakeda/project/study/kmodels/datasets/COVID/images/0.png'

In [53]:
df = pl.concat([df, plus], how="horizontal")

In [54]:
data = df[["path", "new_file_path"]].to_numpy()

In [61]:
for (path, new_path) in data:
    if Path(path).exists()==False:
        print(path)
    os.rename(path, new_path)

In [59]:
df["new_file_path"][0]

'/home/katsujitakeda/project/study/kmodels/datasets/COVID/images/0.png'

In [60]:
df["path"][0]

'/home/katsujitakeda/project/study/kmodels/datasets/COVID/images/2020.01.24.919183-p27-132.png'

In [67]:
new_df = df.select("new_file_name", "target", "mode", "new_file_path")
new_df.columns = ["file_name", "target", "mode", "path"]

In [15]:
df = pl.read_csv("../../datasets/COVID/covid_data_ncc.csv")
df = df.filter(pl.col("file_name")!=14)
df = df.filter(pl.col("file_name")!=15)
df.write_csv("../../datasets/COVID/covid_data_ncc.csv")

In [16]:

# データ拡張のためのtransformを定義
transform = A.Compose([
    A.Resize(224,224),  # 64x64にリサイズ
    A.Normalize(mean=(0.1307,), std=(0.3081,)),  # 平均と標準偏差で正規化
    ToTensorV2()  # テンソルに変換
])

dataset = VGGDataset(csv_path="../../datasets/COVID/covid_data_ncc.csv", mode="train", transform=transform)

In [36]:
csv_name = df["file_name"].to_list()
dir_name = [int(Path(path).name.split(".")[0]) for path in glob("../../datasets/COVID/images/*")]

In [55]:
nums = []
for num in csv_name:
    if num  in dir_name:
        nums.append(num)

In [57]:
df.filter(pl.col("file_name").is_in(nums)).write_csv("../../datasets/COVID/covid_data_ncc.csv")